In [142]:
# Import required modules
import torch
import torchvision
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize, Resize
import matplotlib.pyplot as plt

In [143]:
# Convert to tensors and normalize
transform = Compose([
    Resize((32, 32)),
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="datasets/cifar10",
    train=True,
    download=True,
    transform=transform,
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="datasets/cifar10",
    train=False,
    download=True,
    transform=transform,
)

Files already downloaded and verified
Files already downloaded and verified


In [144]:
# Training and testing functions
def train(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to("cpu"), y.to("cpu")
        optimizer.zero_grad()

        # Compute prediction error
        pred = model(X)
        loss = loss_function(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()

def test(dataloader, model, loss_function):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to("cpu"), y.to("cpu")
            pred = model(X)
            test_loss += loss_function(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return correct, test_loss

In [145]:
# Create data loaders
BATCH_SIZE = 32

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [146]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=30, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=30, out_channels=13, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3),
            nn.Flatten(),
            nn.Linear(208, 120),
            nn.ReLU(),
            nn.Linear(120, 86),
            nn.ReLU(),
            nn.Linear(86, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to("cpu")

print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(30, 13, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=208, out_features=120, bias=True)
    (8): ReLU()
    (9): Linear(in_features=120, out_features=86, bias=True)
    (10): ReLU()
    (11): Linear(in_features=86, out_features=10, bias=True)
    (12): Softmax(dim=1)
  )
)


In [147]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 15
acc, loss = [], []
for t in range(epochs):
    print(f"-------------------------------\nEpoch {t+1}")
    train(train_dataloader, model, loss_function, optimizer)
    temp_acc, temp_loss = test(test_dataloader, model, loss_function)
    acc.append(temp_acc)
    loss.append(temp_loss)
    print(f"Accuracy: {(100*acc[-1]):>0.1f}%, Avg loss: {loss[-1]:>8f}")


# # with open("results/results.txt", "a") as file:
# #     file.write(f"Accuracy: {(100*acc[-1]):>0.1f}%, Avg loss: {loss[-1]:>8f}, Epochs: {epochs}\n{model}\n")

-------------------------------
Epoch 1
Accuracy: 41.1%, Avg loss: 2.047176
-------------------------------
Epoch 2
Accuracy: 45.4%, Avg loss: 2.003630
-------------------------------
Epoch 3
Accuracy: 49.0%, Avg loss: 1.966004
-------------------------------
Epoch 4
Accuracy: 48.6%, Avg loss: 1.971656
-------------------------------
Epoch 5
Accuracy: 55.0%, Avg loss: 1.909602
-------------------------------
Epoch 6
Accuracy: 58.4%, Avg loss: 1.876141
-------------------------------
Epoch 7
Accuracy: 57.3%, Avg loss: 1.886723
-------------------------------
Epoch 8
Accuracy: 61.6%, Avg loss: 1.841258
-------------------------------
Epoch 9
Accuracy: 59.0%, Avg loss: 1.867739
-------------------------------
Epoch 10
Accuracy: 61.1%, Avg loss: 1.846611


In [1]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.plot(np.arange(1, epochs+1), acc, color="blue", marker="o")
ax1.set_title("Accuracy over epochs")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")

ax2.plot(np.arange(1, epochs+1), loss, color="red", marker="o")
ax2.set_title("Loss over epochs")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

In [149]:
# # Saving the model
# model_name = "test_model3"
# torch.save(model.state_dict(), f"models/{model_name}.pth")
# print(f"Saved PyTorch Model State to models/{model_name}.pth")

In [150]:
# # Load the model FAIL
# model_name = "test_model2"
# loaded_model = NeuralNetwork().to("cpu")
# loaded_model.load_state_dict(torch.load(f"models/{model_name}.pth", weights_only=True))
# print("Loaded!\n", loaded_model)